# Notebook for Converting Fine-Tuned Gemma-3n Model to TFLite

This notebook contains the complete process to:
1. Install necessary libraries (including the latest Transformers from source).
2. Authenticate with Hugging Face to download the model.
3. Load your fine-tuned Gemma-3n model and tokenizer.
4. Convert the model to the `.tflite` format using `ai-edge-torch`.
5. Save the converted model and tokenizer assets for use in the Android app.

In [ ]:
# Step 1: Install all required libraries
# We install transformers directly from the GitHub main branch to get support for the newest models like Gemma-3n.
!pip install git+https://github.com/huggingface/transformers.git
!pip install torch accelerate bitsandbytes sentencepiece ai-edge-torch --quiet

print("✅ Libraries installed successfully.")

In [ ]:
# Step 2: Authenticate with Hugging Face (Kaggle Version)
# This cell uses the correct method for Kaggle Secrets.

from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

try:
    user_secrets = UserSecretsClient()
    hf_token = user_secrets.get_secret("HF_TOKEN")
    login(token=hf_token)
    print("✅ Successfully authenticated with Hugging Face.")
except Exception as e:
    print(f"Could not log in. Please ensure you have attached the 'HF_TOKEN' secret via the 'Add-ons' menu. Error: {e}")

In [ ]:
# Step 3: Run the Full Conversion Script

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import ai_edge_torch
import os

# --- Configuration ---
HF_MODEL_PATH = "tamazightdev/v2-gemma-3n-4b-tmz-ft-vllm-merged"
OUTPUT_TFLITE_MODEL = "gemma-3n-4b-tamazight-ft.tflite"
TOKENIZER_OUTPUT_DIR = './tokenizer_assets'

print("--- Starting Model Conversion Process ---")

# --- 1. Load Hugging Face Model and Tokenizer ---
print(f"Loading model and tokenizer from: {HF_MODEL_PATH}")
try:
    # Added trust_remote_code=True which is necessary for brand new architectures
    model = AutoModelForCausalLM.from_pretrained(HF_MODEL_PATH, trust_remote_code=True)
    tokenizer = AutoTokenizer.from_pretrained(HF_MODEL_PATH, trust_remote_code=True)
    print("✅ Model and tokenizer loaded.")

    # It is critical to save the tokenizer's vocabulary for the app.
    os.makedirs(TOKENIZER_OUTPUT_DIR, exist_ok=True)
    tokenizer.save_pretrained(TOKENIZER_OUTPUT_DIR)
    print(f"✅ Tokenizer assets saved to '{TOKENIZER_OUTPUT_DIR}' directory.")

    # --- 2. Define an Example Input ---
    # This helps the converter understand the model's architecture.
    print("Defining an example input for the converter...")
    example_input_text = "Translate to Tamazight: Hello, how are you?"
    inputs = tokenizer(example_input_text, return_tensors="pt")
    example_input = (inputs.input_ids,) # The converter expects a tuple of inputs
    print("✅ Example input created.")

    # --- 3. Convert the Model to TFLite ---
    print("Converting model to TFLite format with INT8 quantization...")
    edge_model = ai_edge_torch.convert(
        model,
        example_input,
        quantization_type=ai_edge_torch.quantization.QuantizationType.INT8
    )
    print("✅ Model successfully converted.")

    # --- 4. Save the Converted Model ---
    with open(OUTPUT_TFLITE_MODEL, "wb") as f:
        f.write(edge_model)

    print("\n--- Conversion Complete! ---")
    print(f"✅ Successfully saved TFLite model to: {OUTPUT_TFLITE_MODEL}")

except Exception as e:
    print(f"\n--- An Error Occurred ---")
    print(f"Error during conversion: {e}")
    print("Please check the model path, your Hugging Face token permissions, and available RAM.")

### Next Steps

After running the cells above, you should have two main outputs in your Kaggle environment's file system (check the file panel on the left):

1.  **`gemma-3n-4b-tamazight-ft.tflite`**: This is your quantized, on-device model.
2.  A folder named **`tokenizer_assets`**: This contains `tokenizer.json` and other necessary files.

You will need to **download both the `.tflite` file and the `tokenizer.json` file** to your local machine to integrate them into your Android project as described in Stage 2 of the instructions.